# Using LASSO Regression for Modeling Fat Sales Data

## Make a copy!

Before getting started, make a copy of this notebook into your own Google Drive:

    Go to: File > Save a copy in Drive

## ⚠️ Autograder ignore sections

This notebook may contain one or more code cells wrapped with the following comment-tags:


The start tag:

```
    #~~ grader-ignore:
```

and the end tag:

```
    #~~ /grader-ignore
```

**These tags must be kept in place and not modified in order for the grader to work properly with your submission file.**

**Important note**

There are no functions to implement. Instead, the grader will be looking at the values of a number of variables that should be assigned. Be sure to:

 * follow the notebook instructions carefully
 * complete all parts that need to be completed
 * name things as instructed / don't change existing variable names


The grader will be looking at your values for all of the following:

 * descriptors
 * top_predictors
 * train_error
 * test_error
 * rsquared_train
 * rsquared_test

Sections 1 & 2 do not require additional code. Read the content of those sections and execute and understand the existing code.

Section 3 has completion sections and requires that you write code to complete the assignment for submission to the grader.

## Section 1. Load and prepare the data

In this first section of the assignment, you will not need to modify any of the code. Read through the section and execute the existing code. Be sure you understand what is happening.

At the end of this section you will have the data loaded into training and testing data sets, ready for analsysis.

### Background

Often in marketing analytics we have too much data to do a simple multiple regression. That is, there are too many possible predictors to consider at once. Multiple regression falls apart in these instances because of [multicollinearity](https://en.wikipedia.org/wiki/Multicollinearity) and because often many variables will be significant, leaving us with no real idea what the few true factors driving the result we want truly are.

Luckily, some newer regression-like approaches have emerged to help us handle this type of fat data (data with tons of possible predictors). This week you will work through a beautiful python implementation of [LASSO](https://en.wikipedia.org/wiki/Lasso_(statistics)) (least absolute shrinkage and selection operator). LASSO is great in that it preforms feature (predictor) variable selection. That is, it automatically selects the most powerful variables, the variables that explain the most variance in our regression, while leaving out those that explain little unique variance.

The dataset you're going to work on is sales data. "# Purchases" is the raw count of sales we saw for a calendar year in a particular are of the U.S. All of the other columns refer to Census data. That is, descriptives about the number of people living in a certain area. We'll be able to look up the census variables to figure out exactly what they mean later, but for now, I want you to focus on the idea that certain Census variables might predict sales. For instance, the number of households in an area that make over 100k a year in annual income might predict the amount of sales for luxury items, such as a Rolex. We're going to perform this analysis to see what Census variables most correlate to sales of our product, which was in this case was [Bobo Bars](https://eatbobos.com/).

For this assignment, use the finalmaster-ratios.csv data file. To avoid the trouble of mounting your Google Drive, you can fetch the data file directly from this URL: https://s3.amazonaws.com/vargo.aprd6342/data/finalmaster-ratios.csv

This is done for you below.

### Imports

In [1]:
import pandas as pd
import pandas
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoLarsCV
import matplotlib.pyplot as plt

### Load the data

Be sure to upload the data file to the root of your Google Drive account and to mount your Google Drive in this notebook.

For compatibility with the grader, do not change the path of the file.

In [4]:
alldata = pd.read_csv("drive/MyDrive/finalmaster-ratios.csv")
alldata.head()

,# Purchases,B01001001,B01001002,B01001003,B01001004,B01001005,B01001006,B01001007,B01001008,B01001009,...,B19001008,B19001009,B19001010,B19001011,B19001012,B19001013,B19001014,B19001015,B19001016,B19001017
0,22,206252,469.226965,31.432422,35.219052,33.628765,20.121017,12.610787,6.734480,6.225394,...,49.409690,53.306757,42.318307,83.167229,89.249208,102.141470,52.872330,36.440765,23.446284,21.197485
1,7,61399,486.538869,22.899396,21.531295,27.036271,16.808091,28.355511,18.192479,13.534422,...,59.231680,50.093078,40.700626,92.612963,117.363344,113.344051,75.774243,33.000508,33.169741,24.792689
2,3,73170,489.859232,28.905289,36.271696,28.235616,21.566216,12.218122,7.243406,7.380074,...,63.996993,47.322923,42.505211,70.420610,90.033143,98.677692,54.703249,20.125056,11.890525,16.537397
3,94,251724,505.585483,32.054949,31.757004,28.102207,18.651380,12.080692,7.035483,7.686991,...,54.790900,48.681562,43.873381,84.717507,112.204444,127.137252,83.019904,43.731067,38.851729,40.427349
4,0,37382,495.586111,25.413301,29.318924,26.162324,19.260607,12.893906,6.580707,7.062222,...,58.883378,51.761414,47.310187,81.902582,93.793717,130.103014,71.982704,36.118530,31.603714,19.648989


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


There are 190 variables in this dataset:

 * One outcome variable "# Purchases", and
 * 189 possible predictors.

Create a list of all the predictors you're going to feed into the LassoLarsCV model. Assuming you've loaded your data into a pandas dataframe and named that dataframe alldata, you can get a list of your variables easily:

In [5]:
all_variables = list(alldata.columns.values)

For the sake of this homework, we know that the first 8 variables aren't valid predictors of what we're interested in. The first variable in the list is the outcome variable, and the next 7 are repetitive of other variables, so we need to exclude them. Go ahead and remove those items from the list.

In [6]:
all_variables = all_variables[8:]

In [7]:
all_variables[0:5]

['B01001008', 'B01001009', 'B01001010', 'B01001011', 'B01001012']

In [8]:
len(all_variables)

182

Now, let's get our columns ready in pandas. Your target variable should be:


In [9]:
#load predictors into dataframe
predictors = alldata[all_variables]

In [10]:
#load target into dataframe
target = alldata['# Purchases']   

### Split the data into train and test sets

Remember, you have 182 predictors and one thing you're predicting.

Next, with most advanced regression models, we need to split the data into training and test sets. We train the model on the training set, and we test the model with the test set, to see how well the model actually performs. All this line of code is really doing is splitting the data (by rows) where we use 70% of the data to train, and 30% of the data to test. This method selects the data randomly, as to avoid any biases:


In [11]:
# split data into train and test sets, with 30% retained for test

pred_train, pred_test, tar_train, tar_test = train_test_split(predictors, target, test_size=.3, random_state=123) 

In [12]:
pred_train.head()

,B01001008,B01001009,B01001010,B01001011,B01001012,B01001013,B01001014,B01001015,B01001016,B01001017,...,B19001008,B19001009,B19001010,B19001011,B19001012,B19001013,B19001014,B19001015,B19001016,B19001017
309,9.023455,9.727580,25.827791,35.673712,35.082010,32.744787,30.756668,36.123405,37.874843,34.283212,...,50.537432,50.537432,47.085104,98.159242,117.770783,133.103178,79.693642,38.381758,29.547861,27.807192
414,3.893270,7.439369,17.829688,28.740763,29.658285,25.715419,30.327828,29.261519,34.121906,28.492784,...,50.298223,47.654184,48.576523,73.541167,85.285618,71.757978,49.006948,15.126360,9.407858,19.307631
691,4.916925,6.897677,18.199613,32.996994,30.293850,30.060821,29.781185,33.602871,36.306014,31.808543,...,57.632303,73.600489,35.668400,84.307128,133.374121,142.918324,90.302845,35.668400,31.141022,17.803610
669,13.435004,11.656704,25.343091,29.644526,28.192094,26.004134,32.353872,30.966613,31.339031,32.009385,...,47.653262,48.649901,40.120524,79.870205,94.912504,83.277321,49.994206,17.406420,18.912968,17.452776
534,8.665299,10.808659,32.425978,40.800392,33.451728,26.133684,29.838636,26.179614,33.359870,25.858110,...,62.869697,50.870373,49.349332,79.558898,103.684299,117.415920,41.786378,25.266182,16.984959,12.295082


We now have four new pandas data frames:

 1. pred_train, the predictors training set 
 2. pred_test, the predictors test test
 3. target_train, the target training set
 4. tar_test, the target test set
 
We'll feed these datasets to our model in the next step.

## Section 2

You should now have the data loaded as described above.

### Build the model

Build a [LassoLarsCV](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoLarsCV.html) model with the following parameters:

  * cv = 10 (this performs a 10-fold cross validation, to make sure our results aren't do to random ordering of the data)
  * precompute=False (precompute is not necessary)

In [13]:
model = LassoLarsCV(cv=10, precompute=False)

Then fit the newly created model with the training data.

To do this, [use .fit() on your newly created model object](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoLarsCV.html#sklearn.linear_model.LassoLarsCV.fit). Remember what X and y mean from doing regression?

Note: The following code cell produces a large number of Convergence Warnings which may be safely ignored for the purposes  of this assignment.

In [14]:
model.fit(pred_train, tar_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:138: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsCV())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:660: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 5 iterations, i.e. alpha=1.496e+00, with an active set of 5 regressors, and the smallest cholesky piv

LassoLarsCV(cv=10, precompute=False)

Congrats. You've just built your first LASSO model. Next, let's see what variables were significant.

### Build coefficient chart

Next, let's extract the coefficients from the model.

In [15]:
pd.options.display.float_format = '{:.5f}'.format
predictors_model = pd.DataFrame(all_variables)
predictors_model.columns = ['label']
predictors_model['coeff'] = model.coef_
predictors_model

,label,coeff
0,B01001008,0.00000
1,B01001009,0.00000
2,B01001010,0.00000
3,B01001011,0.00000
4,B01001012,0.00000
...,...,...
177,B19001013,0.00000
178,B19001014,0.00000
179,B19001015,0.00000
180,B19001016,0.00000


To build the coefficients chart of final predictors, filter the predictors for coefficient values greater than 0. This is done for you here:

In [16]:
final_predictors = predictors_model[predictors_model['coeff'] > 0]
final_predictors

,label,coeff
6,B01001014,0.85586
28,B01001036,2.50535
29,B01001037,0.88929
30,B01001038,1.53163
46,B02001005,0.41253
96,B13014026,0.48004
97,B13014027,0.69786
113,B13016001,875104587.39904
181,B19001017,1.48344


The variable final_predictors should now be a DataFrame of the 9 labels with coefficients greater than zero.

## Section 3

Now we know which variables most predicted sales. Because of the aforementioned feature section feature of LASSO, any coefficients that are non-zero are significant, and thus variables that truly predict unique amounts of sales. The higher the value of the coefficient, the steeper the relationship between sales and that variable.

### Finding meaning in the data

Let's figure out **what these Census variables actually mean**. To do this, we're going to perform a simple hack, and have you Google the variable name + "Census", [here's an example](https://www.google.com/search?q=census+B01001036&oq=census+B01001036&aqs=chrome..69i57.4071j1j4&sourceid=chrome&ie=UTF-8).  Click on the first result from socialexplorer.com. For this example, I'd report: **Females aged 30 to 34 Years**.

To record your findings, create a dictionary of keys that are named according to the census codes. Assign each key the string value of the description of that code. Do this for all of the codes in the coefficient chart that displayed above. The code from the Google search example above has been completed for you as an example:

In [17]:
descriptors = {
    'B01001036': 'Females aged 30 to 34 years',
    # Complete the remaining assignments from the codes in the positive
    # coefficients chart. Be sure to name the keys exactly the same as the codes.
    'B01001014': 'Males aged 40 to 44 years',
    'B01001037': 'Females aged 35 to 39 years',
    'B01001038': 'Females aged 40 to 44 years',
    'B02001005': 'Asian',
    'B13014026': 'Females with a bachelors degree aged 15 to 50 years who had a birth in the past 12 months',
    'B13014027': 'Females with a graduate/professional degree aged 15 to 50 years who had a birth in the past 12 months',
    'B13016001': 'Females aged 15 to 50 years who had a birth in the past 12 months',
    'B19001017': 'Households with over $200000 household income'
}

### Choosing the top predictors

If I had to report only two census variables to my boss that most steeply predicted sales, what would those be? To answer this question, create a dataframe called top_predictors which contains only the top 2 rows of final_predictors by coefficient.

Pandas [sort_values](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html) will be useful here.


In [20]:
# Assign top_predictors to a frame of the top 2 labels. The easiest way to do
# this is probably to reverse sort final_predictors by coeff and take the [:2]
# slice of the results

# complete this by setting top_predictors:

#sorting values by the coeff value, then sorting it in descending order, then selecting the top two
top_predictors = final_predictors.sort_values(by=['coeff'], ascending = False)[0:2]

top_predictors


,label,coeff
113,B13016001,875104587.39904
28,B01001036,2.50535


In [21]:
#~~ grader-ignore:
top_predictors.label.values
#~~ /grader-ignore


array(['B13016001', 'B01001036'], dtype=object)

If you did all of the above correctly, you should get a simple report below that you can copy-paste into an email to your boss:

In [22]:
#~~ grader-ignore:
print(f"""
In areas where there are { descriptors[top_predictors.label.values[0]] } and 
{ descriptors[top_predictors.label.values[1]]}, we sell more Bobo Bars.
""")
#~~ /grader-ignore


In areas where there are Females aged 15 to 50 years who had a birth in the past 12 months and 
Females aged 30 to 34 years, we sell more Bobo Bars.



### Mean squared error

Next, lets take a look at the mean squared error for the training and training set:

In [23]:
from sklearn.metrics import mean_squared_error

train_error = mean_squared_error(tar_train, model.predict(pred_train))
train_error

22025.457128540853

Do the same thing as above, but for the test data.

In [24]:
# create a variable called test_error with is the mean squared error of the test data

# Complete this by setting test_error


test_error = mean_squared_error(tar_test, model.predict(pred_test))

Compare the test and training results above and consider the following questions: Are the training and text set mean squared errors similar? What does that mean practically? Think back to your stats class, or Google it!

### r-squared

Next, let's see what our R-squared is for the training set: 

 


In [25]:
rsquared_train = model.score(pred_train,tar_train)
rsquared_train

0.24002329299858283

In [26]:
# create a variable called rsquared_test that is the r-square score for the test data

# Complete this by setting rsquared_test
rsquared_test = model.score(pred_test, tar_test )

Compare the r-squared results, and consider the following questions for thought and discusssion:

If your boss asked, "How well does Census data, overall, predict sales?" What would you say? Why?

### Find the y-intercept

Finally, let's see what our y-intercept is, so we can interpret what our baseline sales number looks like, all things considered.

In [27]:
y_intercept = model.intercept_
y_intercept

22.196684711205144

What is our baseline sales number? What does that mean,  practically? Think back to what y-intercepts mean in regression models.